In [7]:
import tensorflow as tf
import utils
utils.mount_src()
import data_loading

CONFIG = utils.load_config()
IMAGES_PATH = CONFIG["images_path"]
DATA_PATH = IMAGES_PATH + "/1900"
BATCH_SIZE = 32
INPUT_SHAPE = (224, 224, 3)
EPOCHS = 10

EXAMPLE_IMAGE = DATA_PATH + "/194028115.jpg"
METADATA_CSV = DATA_PATH + "/metadata.csv"
IMAGE_ID = "194028115"

In [9]:
images, labels = data_loading.__load_data_for_year(DATA_PATH)
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_split = int(0.8 * len(images))
train_dataset = dataset.take(train_split)
val_dataset = dataset.skip(train_split)

In [10]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras import Model


def create_pretrained_model(input_shape: tuple[int, int]) -> Model:
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation="relu"),
        #layers.Dropout(0.5),
        layers.Dense(3)
    ])
    return model

In [ ]:
model = create_pretrained_model(INPUT_SHAPE)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS
)
history

In [ ]:
import matplotlib.pyplot as plt
mae = history.history["mae"]
loss = history.history["loss"]

epochs = range(1, len(mae) + 1)
plt.plot(epochs, mae)
plt.figure()
plt.plot(epochs, loss)

In [ ]:
import numpy as np
from tensorflow.keras.utils import img_to_array, load_img

def load_image_to_shape(file_path: str, target_size: tuple[int, int]) -> np.ndarray:
    img = load_img(file_path, target_size=target_size)
    arr = img_to_array(img) / 255.0
    return np.expand_dims(arr, axis=0) # Adds batch dimension (necessary)

img = load_image_to_shape(EXAMPLE_IMAGE, INPUT_SHAPE)
pred = model.predict(img)
pred

In [31]:
# Descaling
from sklearn.preprocessing import MinMaxScaler
import polars as pl

df = pl.read_csv(METADATA_CSV)
df = df.unique(["id", "year", "latitude", "longitude"]).select(["year", "latitude", "longitude"])

labels = []
for r in df.iter_rows():
    labels.append(r)
labels = np.array(labels)

scaler = MinMaxScaler()
labels[:, 1:] = scaler.fit_transform(labels[:, 1:])

In [ ]:
descaled_pred = scaler.inverse_transform(pred[:, 1:])
descaled_pred